# Apache Spark DataFrames Project ~ P.M. Kuria

## 1. Defining the Question

### a) Specifying the Data Analysis Question

As a Data professional, you need to perform an analysis by answering questions about some stock market data on Safaricom from the years 2012-2017.

### b) Defining the Metric for Success

Measure of success will be measured by atttempting to asnwer below questions: 

1. What day had the Peak High in Price?
2. What is the mean of the Close column?
3. What is the max and min of the Volume column?
4. How many days was the Close lower than 60 dollars?
5. What percentage of the time was the High greater than 80 dollars?
6. What is the Pearson correlation between High and Volume?
7. What is the max High per year?
8. What is the average Close for each Calendar Month?

### c) Understanding the context 

Safaricom PLC is the leading telecommunication company in East Africa. Its purpose is to transform lives by connecting people to people, people to opportunities and people to information. It keeps over 42 million customers connected and play a critical role in the society, supporting over one million jobs both directly and indirectly while our total economic value was estimated at KES 362 Billion ($ 3.2 billion) for the 12 months through March 2021. It is listed on the Nairobi Securities Exchange (NSE) and with annual revenues of close to KES 298 Billion ($2.5 billion) as at March 2022. It is founded in 1997 as a fully owned subsidiary of Telkom Kenya before a 40 percent acquisition by Vodafone Group PLC in May 2000, and a public offering of 25 percent shares through the NSE in 2008. Under the management of Vodafone Group PLC, it welcomed Michael Joseph, as our first CEO, a few months later in July of 2000. He led the company’s growth to accommodate 16.71 million subscribers from the previous 20,000, largely owing to innovative products like M-PESA in 2007

### d) Recording the Experimental Design

1. Data Importation and Exploration.
2. Data Preparation.
3. Data Analysis.

### e) Data Relevance

The dataset used is indisputable, since it is available publicly online and anyone can use the same data to challenge any conclusion arrived at on the analysis performed herein.

## 2. Data Importation and Exploration


In [2]:
# install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=c8c0e2a7742198ba38b9f05d75a297027c4d67073489380f19b227929d9e8da7
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
# Importing our libraries
# pyspark
#
from pyspark.sql import SparkSession
from pyspark.sql.functions import format_number, mean, max, min, corr, count, avg
from pyspark.sql.functions import year, month

In [12]:
# Start a spark session
spark = SparkSession.builder.appName("StockAnalysis").getOrCreate()

# Load the stock file and infer the data types
df = spark.read.csv("saf_stock.csv", header=True, inferSchema=True)

In [13]:
# Determine the column names
columns = df.columns
print("Columns: ", columns)

Columns:  ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']


In [7]:
# Make observations about the schema
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [8]:
# Show the first 5 rows
df.show(5)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [9]:
# Use the describe method to learn about the data frame
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## 3. Data Preparation

In [22]:
# Format all the numeric data to 2 decimal places
for c_name, c_type in df.dtypes:
    if c_type in ('double', 'float'):
        df = df.withColumn(c_name, format_number(c_name, 2))

df.show(5)

+-------------------+-----+-----+-----+-----+--------+---------+
|               Date| Open| High|  Low|Close|  Volume|Adj Close|
+-------------------+-----+-----+-----+-----+--------+---------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|    52.62|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|    52.08|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|12768200|    51.83|
|2012-01-06 00:00:00|59.42|59.45|58.87|59.00| 8069400|    51.46|
|2012-01-09 00:00:00|59.03|59.55|58.92|59.18| 6679300|    51.62|
+-------------------+-----+-----+-----+-----+--------+---------+
only showing top 5 rows



In [24]:
# Create a new data frame with a column called HV Ratio
df = df.withColumn("HV Ratio", df["High"]/df["Volume"])
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'HV Ratio']


## 4. Data Analysis

In [25]:
# What day had the Peak High in Price?
df.orderBy(df["High"].desc()).show(1)

+-------------------+-----+-----+-----+-----+-------+---------+--------------------+
|               Date| Open| High|  Low|Close| Volume|Adj Close|            HV Ratio|
+-------------------+-----+-----+-----+-----+-------+---------+--------------------+
|2015-01-13 00:00:00|90.80|90.97|88.93|89.31|8215400|    83.83|1.107310660466928E-5|
+-------------------+-----+-----+-----+-----+-------+---------+--------------------+
only showing top 1 row



In [26]:
# What is the mean of the Close column?
mean_close = df.agg(mean(df["Close"])).collect()[0][0]
print("Mean of Close column: ", mean_close)

Mean of Close column:  72.38844992050863


In [27]:
# What is the max and min of the Volume column?
max_volume = df.agg(max(df["Volume"])).collect()[0][0]
min_volume = df.agg(min(df["Volume"])).collect()[0][0]
print("Max of Volume column: ", max_volume)
print("Min of Volume column: ", min_volume)

Max of Volume column:  80898100
Min of Volume column:  2094900


In [28]:
# How many days was the Close lower than 60 dollars?
days_close_lt_60 = df.filter(df["Close"] < 60).count()
print("Days Close < 60: ", days_close_lt_60)

Days Close < 60:  81


In [29]:
# What percentage of the time was the High greater than 80 dollars?
high_gt_80 = df.filter(df["High"] > 80).count() / df.count() * 100
print("% High > 80: ", high_gt_80)

% High > 80:  8.426073131955485


In [30]:
# What is the Pearson correlation between High and Volume?
correlation = df.agg(corr("High", "Volume")).collect()[0][0]
print("Pearson Correlation: ", correlation)

Pearson Correlation:  -0.33843260582148915


In [31]:
# What is the max High per year?
df_year = df.groupBy(year(df["Date"])).agg(max(df["High"]))
df_year.show()

+----------+---------+
|year(Date)|max(High)|
+----------+---------+
|      2012|    77.60|
|      2013|    81.37|
|      2014|    88.09|
|      2015|    90.97|
|      2016|    75.19|
+----------+---------+



In [32]:
# What is the average Close for each Calendar Month?
df_month = df.groupBy(month(df["Date"])).agg(avg(df["Close"]))
df_month.show()

+-----------+-----------------+
|month(Date)|       avg(Close)|
+-----------+-----------------+
|         12|72.84792452830189|
|          1|71.44801980198022|
|          6|72.49537735849057|
|          3|71.77794392523363|
|          5|72.30971698113206|
|          9|72.18411764705883|
|          4|72.97361904761907|
|          8|73.02981818181819|
|          7|74.43971962616824|
|         10|71.57854545454546|
|         11|72.11108910891085|
|          2|71.30680412371134|
+-----------+-----------------+



### a). Did we have the right data?

Yeah, to answer the questions given.

### b). Do we need other data to answer our question?

For the required analysis this data is sufficient for now.

### c). Did we have the right question?

Yes as to meet the brief.